<a href="https://colab.research.google.com/github/Reine2001/Federated-learning-based_Chatbot/blob/main/DistilBERT_Juridique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DistilBERT Spécialisé domaine juridique**

**Installation des dépendances nécessaires**

In [ ]:
!pip install transformers tensorflow tensorflow-federated


**Import des bibliothèques**

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from transformers import DistilBertTokenizer, TFDistilBertModel
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel



**Chargement des données et prétraitement**

In [ ]:
# Charger le tokenizer et le modèle DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
# Chemins des fichiers clients
filepaths = {
    'Droits_humains': 'Droits_humains.xlsx',
    'Police_Judiciaire': 'Police_Judiciaire.xlsx',
    'Police_Nationale': 'Police_Nationale.xlsx',
    'Securite': 'Securite.xlsx'
}

In [ ]:
# Chargement des fichiers en tant que DataFrames pandas
clients_data = {}
for client, filepath in filepaths.items():
    clients_data[client] = pd.read_excel(filepath)


In [ ]:
# Prétraitement des données
def preprocess_data(data):
    # Supposons que chaque fichier a une colonne 'article' avec les textes légaux
    data['article_clean'] = data['Article'].str.strip().str.lower()
    return data
for client, data in clients_data.items():
    clients_data[client] = preprocess_data(data)

In [ ]:
# Fonction pour générer les embeddings avec DistilBERT
def generate_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='tf', padding=True, truncation=True, max_length=128)
    outputs = distilbert_model(inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]  # Utiliser la première position comme embedding
    return embeddings


In [ ]:
# Fonction pour générer les embeddings avec DistilBERT
def generate_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='tf', padding=True, truncation=True, max_length=128)
    outputs = distilbert_model(inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]  # Utiliser la première position comme embedding
    return tf.cast(embeddings, dtype=tf.float32)  # Convertir les embeddings en float32

# Générer les embeddings pour chaque client
for client, data in clients_data.items():
    articles = data['article_clean'].tolist()
    embeddings = generate_embeddings(articles)

    # Convertir les embeddings en liste de float32 pour chaque article
    clients_data[client]['embedding'] = embeddings.numpy().astype(np.float32).tolist()


In [ ]:
# Préparer les datasets pour l'apprentissage fédéré
# Préparer les datasets pour chaque client avec conversion en float32
def preprocess_federated_data(client_data):
    inputs = np.vstack(client_data['embedding'].values).astype(np.float32)  # Convertir les embeddings en float32
    targets = np.random.rand(len(inputs), 1).astype(np.float32)  # Convertir les cibles en float32
    dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    return dataset.batch(32)

clients_tf_data = [preprocess_federated_data(data) for data in clients_data.values()]


In [ ]:
# Vérification des types après la conversion
for client, data in clients_data.items():
    print(f"Client {client} - Embeddings type: {data['embedding'].dtype}")


Client Droits_humains - Embeddings type: object
Client Police_Judiciaire - Embeddings type: object
Client Police_Nationale - Embeddings type: object
Client Securite - Embeddings type: object


In [ ]:
# Modèle pour l'apprentissage fédéré
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(768,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])
    return model


In [ ]:
# Fonction pour définir le modèle pour TensorFlow Federated
def model_fn():
    keras_model = create_model()
    loss = tf.keras.losses.MeanSquaredError()
    metrics = [tf.keras.metrics.MeanSquaredError()]

    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=(tf.TensorSpec(shape=[None, 768], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.float32)),
        loss=loss,
        metrics=metrics
    )


In [ ]:
# Configuration de l'apprentissage fédéré
def client_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=0.01)

def server_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=1.0)

iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

In [ ]:
def evaluate_client_model(state, client_dataset):
    """
    Évaluer le modèle pour un client donné en calculant la perte (loss).
    """
    keras_model = create_model()  # Créer un modèle Keras similaire au modèle TFF
    model_weights = iterative_process.get_model_weights(state)

    # Charger les poids du modèle global dans le modèle Keras
    keras_model.set_weights(model_weights.trainable)

    # Calculer la perte (loss) sur le dataset du client
    loss_fn = tf.keras.losses.MeanSquaredError()

    total_loss = 0
    num_batches = 0

    for batch in client_dataset:
        inputs, targets = batch
        predictions = keras_model(inputs)
        loss = loss_fn(targets, predictions)
        total_loss += loss
        num_batches += 1

    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    return avg_loss

In [ ]:
# Fonction pour afficher les métriques par client
def print_client_metrics(state, client_data, round_num):
    print(f"Round {round_num} - Metrics per client:")
    client_metrics = []

    for i, client_dataset in enumerate(client_data):
        # Évaluer le modèle localement pour chaque client
        avg_loss = evaluate_client_model(state, client_dataset)
        client_metrics.append(avg_loss)

        print(f"Client {i + 1} metrics (average loss): {avg_loss.numpy()}")

    return client_metrics


In [ ]:
# Fonction d'entraînement fédéré avec affichage des métriques par client
def train_federated_model(state, client_data, num_rounds):
    for round_num in range(1, num_rounds + 1):
        print(f"--- Starting round {round_num} ---")

        # Afficher les métriques locales par client
        client_metrics = print_client_metrics(state, client_data, round_num)

        # Effectuer l'étape suivante de l'apprentissage fédéré
        state, global_metrics = iterative_process.next(state, client_data)

        # Afficher les métriques globales après agrégation
        print(f"Global metrics after round {round_num}: {global_metrics}")
        print("\n")

    return state


In [ ]:
# Initialisation de l'état du processus
state = iterative_process.initialize()

# Nombre de rounds d'apprentissage
num_rounds = 10

# Exécuter l'entraînement fédéré avec affichage des métriques par client et globales
state = train_federated_model(state, clients_tf_data, num_rounds)


--- Starting round 1 ---
Round 1 - Metrics per client:
Client 1 metrics (average loss): 0.2775785028934479
Client 2 metrics (average loss): 0.23096705973148346
Client 3 metrics (average loss): 0.30938273668289185
Client 4 metrics (average loss): 0.2507466971874237
Global metrics after round 1: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_squared_error', 0.39847952), ('loss', 0.39847952), ('num_examples', 2647), ('num_batches', 85)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


--- Starting round 2 ---
Round 2 - Metrics per client:
Client 1 metrics (average loss): 0.08431078493595123
Client 2 metrics (average loss): 0.09324019402265549
Client 3 metrics (average loss): 0.08561871945858002
Client 4 metrics (average loss): 0.07997776567935944
Global metrics after round 2: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDi

In [ ]:
# Fonction pour sauvegarder les poids du modèle global après l'entraînement fédéré
def save_global_model(state, filepath):
    model_weights = iterative_process.get_model_weights(state)
    # Créer un modèle Keras similaire à celui utilisé dans le processus fédéré
    keras_model = create_model()
    # Charger les poids du modèle global dans le modèle Keras
    keras_model.set_weights(model_weights.trainable)
    # Sauvegarder le modèle Keras (ou simplement ses poids)
    keras_model.save_weights(filepath)
    print(f"Modèle fédéré sauvegardé dans : {filepath}")

# Sauvegarder le modèle global après l'entraînement
save_global_model(state, '/content/distilbert_juridique.h5')


Modèle fédéré sauvegardé dans : /content/distilbert_juridique.h5


# **Indexation dans Elasticsearch**

**Imports des bibliothèques**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import requests


In [ ]:

#Chargement des fichiers pour les Droits_humains
Droits_humains=pd.read_excel("/content/Droits_humains.xlsx")

#Chargement des fichiers pour la Police_Judiciaire
Police_Judiciaire=pd.read_excel("/content/Police_Judiciaire.xlsx")

#Chargement des fichiers pour la Police_Nationale
Police_Nationale=pd.read_excel("/content/Police_Nationale.xlsx")

#Chargement des fichiers pour la Securite
Securite=pd.read_excel("/content/Securite.xlsx")


In [ ]:
# Fonction pour recréer le modèle et charger les poids du modèle fédéré
def load_federated_model(filepath):
    # Recréer le modèle utilisé pour l'apprentissage fédéré
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(768,)),  # Taille des embeddings
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])
    # Charger les poids sauvegardés
    model.load_weights(filepath)
    print(f"Modèle fédéré chargé depuis : {filepath}")
    return model

# Charger le modèle fédéré sauvegardé
federated_model = load_federated_model('/content/distilbert_juridique.h5')

Modèle fédéré chargé depuis : /content/distilbert_juridique.h5


In [ ]:

# Fonction pour découper un texte en morceaux avec une fenêtre glissante
def sliding_window_tokenize(text, max_len=512, stride=256):
    tokens = tokenizer(text, truncation=False, return_tensors='tf')['input_ids'][0]
    token_chunks = []

    for i in range(0, len(tokens), stride):
        chunk = tokens[i:i + max_len]
        token_chunks.append(chunk)

        if len(chunk) < max_len:
            break  # Si le morceau est plus petit que la taille maximale, on arrête

    return token_chunks


In [ ]:
from transformers import BertTokenizer, TFBertModel

# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Fonction pour générer les embeddings avec BERT (taille 768)
def generate_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='tf', padding=True, truncation=True, max_length=512)
    outputs = bert_model(inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]  # Utiliser la première position comme embedding
    return embeddings  # Embeddings de taille (batch_size, 768)

# Utiliser cette fonction dans votre pipeline à la place de celle avec le modèle fédéré directement
def generate_embeddings_with_federated_model(text_list):
    all_embeddings = []

    for text in text_list:
        # Obtenir les embeddings de BERT (taille 768)
        embedding = generate_bert_embeddings([text])
        all_embeddings.append(embedding.numpy())

    return np.array(all_embeddings)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# URL ngrok
ngrok_url = 'https://5dc0-197-239-108-118.ngrok-free.app'

# Fonction pour indexer les embeddings dans ElasticSearch
def index_embeddings_in_elasticsearch(articles, embeddings, index_name, ngrok_url):
    for i, (article, embedding) in enumerate(zip(articles, embeddings)):
        doc = {
            'article_id': f'article_{i}',
            'text': article,
            'embedding': embedding.tolist()  # Transformer en liste pour JSON
        }
        response = requests.post(f'{ngrok_url}/{index_name}/_doc/',
                                 headers={"Content-Type": "application/json"},
                                 data=json.dumps(doc))
        if response.status_code != 201:
            print(f"Erreur d'indexation pour l'article {i}: {response.text}")
        else:
            print(f"Article {i} indexé avec succès.")

# Fonction pour traiter les données d'une institution
def process_institution(df, institution_name, ngrok_url):
    articles = []

    for index, row in df.iterrows():
        # Formater le texte de l'article avec le numéro de l'article et la loi
        formatted_text = f"Article {row['Num_Article']} Loi {row['loi']} : {row['Article']}"
        articles.append(formatted_text)

    # Générer les embeddings pour chaque article avec le modèle fédéré
    embeddings = generate_embeddings_with_federated_model(articles)

    # Indexer les articles formatés et leurs embeddings dans Elasticsearch
    index_embeddings_in_elasticsearch(articles, embeddings, f"{institution_name.lower()}_embedding", ngrok_url)

# Traiter les données pour chaque institution
process_institution(Droits_humains, 'Droits_humains', ngrok_url)
process_institution(Police_Judiciaire, 'Police_Judiciaire', ngrok_url)
process_institution(Police_Nationale, 'Police_Nationale', ngrok_url)
process_institution(Securite, 'Securite', ngrok_url)


**Recuperation d'information**

Module de classification

In [ ]:
def classify_question(query_text):

    droits_humains_keywords = ["handicap", "indigence", "droits humains", "protection des personnes handicapées", "inclusion sociale", "personnes vulnérables"]
    police_judiciaire_keywords = ["violence", "crimes", "délits", "prévention de la violence", "justice pénale", "enquête judiciaire", "procédure pénale"]
    police_nationale_keywords = ["académie de police", "formation policière", "forces de l'ordre", "admission stagiaires", "missions de police", "sécurité publique"]
    securite_keywords = ["gardiennage", "sociétés privées de sécurité", "sécurité nationale", "sécurité privée", "protection des biens", "responsabilité professionnelle"]


    if any(keyword in query_text.lower() for keyword in droits_humains_keywords):
        return "droits_humains_embeddings"
    elif any(keyword in query_text.lower() for keyword in police_judiciaire_keywords):
        return "police_judiciaire_embeddings"
    elif any(keyword in query_text.lower() for keyword in police_nationale_keywords):
        return "police_nationale_embeddings"
    elif any(keyword in query_text.lower() for keyword in securite_keywords):
        return "securite_embeddings"
    else:

        return "general_embeddings"


Module de recherche

In [ ]:
def search_combined_in_elasticsearch(query_text, ngrok_url, index_name, weight_text=0.7, weight_vector=0.3):
    # Générer l'embedding pour la requête
    tokens = tokenizer(query_text, truncation=True, return_tensors='tf')['input_ids']
    inputs = {'input_ids': tokens}
    outputs = model(inputs)
    query_embedding = outputs.last_hidden_state[:, 0, :].numpy().flatten()

    # Construire la requête Elasticsearch
    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "text": query_text  # Recherche textuelle classique
                        }
                    },
                    {
                        "script_score": {
                            "query": {
                                "match_all": {}  # Appliquer la recherche vectorielle sur tous les documents
                            },
                            "script": {
                                "source": f"cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                "params": {
                                    "query_vector": query_embedding.tolist()
                                }
                            }
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        }
    }

    # Exécuter la requête
    response = requests.post(f'{ngrok_url}/{index_name}/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))
    if response.status_code == 200:
        results = response.json()
        print(f"Résultats de la recherche combinée pour '{query_text}':")
        for hit in results['hits']['hits']:
            print(f"Article ID: {hit['_source']['article_id']}, Score: {hit['_score']}, Text: {hit['_source']['text']}")
    else:
        print(f"Erreur lors de la recherche: {response.text}")


**Langchain et streamlit**

Voir app.py